Jordi Tudela - AG2

Actividad Guiada 2

URL: https://github.com/bar0net/03MAIR---Algoritmos-de-Optimizacion/blob/master/AG2/JordiTudela-AG2.ipynb

Problema de las parejas más cercanas para 1D y 2D resulta en actividad 1: https://github.com/bar0net/03MAIR---Algoritmos-de-Optimizacion/blob/master/AG1/JordiTudela-AG1.ipynb

## Paseo por el río

In [1]:
# Para hacerme más sencilla la copia de datos, uso una lista de tuplas
# donde se especifica en cada tupla el punto de origen, el punto de
# destino y el coste (en ese orden)
tarifas = [
    (0,1,5),
    (0,3,3),
    (0,2,4),
    (1,6,11),
    (1,3,2),
    (1,4,3),
    (3,4,5),
    (3,5,6),
    (3,6,9),
    (2,3,1),
    (2,5,4),
    (2,6,10),
    (4,6,4),
    (5,6,3)
]

matrizTarifas = [[float("Inf") for _ in range(7)] for _ in range(7)]
for i,j,v in tarifas:
    matrizTarifas[i][j] = v
    
print(matrizTarifas)

[[inf, 5, 4, 3, inf, inf, inf], [inf, inf, inf, 2, 3, inf, 11], [inf, inf, inf, 1, inf, 4, 10], [inf, inf, inf, inf, 5, 6, 9], [inf, inf, inf, inf, inf, inf, 4], [inf, inf, inf, inf, inf, inf, 3], [inf, inf, inf, inf, inf, inf, inf]]


### Búsqueda por profundidad

Se exploran todos los caminos del grafo entre el punto inicial y el punto final.

In [2]:
# Depth First Search
# PRE: El grafo definido por matrizTarifas no contiene ciclos
def DFS_Tarifas(matrizTarifas, inicio=0, fin=6):
    explore = [(0,[inicio])]
    best = None
    
    while len(explore) > 0:
        acc_cost, path = explore.pop(0)
        
        # Reached desired node and it is better than
        # what we already found
        if path[-1] == fin and (best == None or acc_cost < best[0]): 
            best = (acc_cost, path)
            continue
            
        for i, value in enumerate(matrizTarifas[path[-1]]):
            if value == float("Inf"):
                continue
                
            explore.append( (acc_cost+value, path + [i]) )
        
    return best
%timeit DFS_Tarifas(matrizTarifas)
DFS_Tarifas(matrizTarifas)

65.1 µs ± 1.98 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


(11, [0, 2, 5, 6])

### Búsqueda por profundidad con "salida temprana"

Una posible mejora que podemos hacer a DFS es evitar explorar ramas que no tienen capacidad para mejorar el mejor resutrado registrado hasta ese momento.

In [3]:
# Depth First Search with preemtive exits
# PRE: El grafo definido por matrizTarifas no contiene ciclos
def DFS_Tarifas2(matrizTarifas, inicio=0, fin=6):
    explore = [(0,[inicio])]
    best = None
    
    while len(explore) > 0:
        acc_cost, path = explore.pop(0)
        
        # Reached desired node and it is better than
        # what we already found
        if path[-1] == fin and (best == None or acc_cost < best[0]): 
            best = (acc_cost, path)
            continue
            
        for i, value in enumerate(matrizTarifas[path[-1]]):
            if value == float("Inf"):
                continue
            
            # Avoid adding non-improving paths
            if best == None or acc_cost+value < best[0]:
                explore.append( (acc_cost+value, path + [i]) )
        
    return best
%timeit DFS_Tarifas2(matrizTarifas)
DFS_Tarifas2(matrizTarifas)

54.5 µs ± 1.52 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


(11, [0, 2, 5, 6])

### Dijkstra

Dijstra nos permite refinar más el método de búsqueda guardando la distancia más corta a cada nodo explorado y recorriendo el árbol priorizando expandir las ramas cuya distancia (descubierta en ese momento) sea menor. Eso nos permite asegurar que la primera vez que alcancemos la meta, lo habremos hecho por el camino más corto.

Este método tambiém admite que haya ciclos en el camino.

In [4]:
def Dijkstra(matrizTarifas, inicio = 0, fin = 6):
    m = len(matrizTarifas)
    
    pesos = [float("Inf") for _ in range(m)]
    previo = [None for _ in range(m)]
    pesos[inicio] = 0
    
    explorar = [0]
    
    while len(explorar) > 0:
        explorar = sorted(explorar, key = lambda x : pesos[x], reverse=True)
        
        current = explorar.pop(0)
        
        for i,x in enumerate(matrizTarifas[current]):
            if x == float("Inf"):
                continue
                
            if pesos[current] + x < pesos[i]:
                pesos[i] = pesos[current] + x
                previo[i] = current
                explorar.append(i)
    
    ruta = [fin]
    while ruta[-1] != inicio:
        ruta.append(previo[ruta[-1]])
    
    return pesos[fin], list(reversed(ruta))
    
%timeit Dijkstra(matrizTarifas)
Dijkstra(matrizTarifas)

37.9 µs ± 1.67 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


(11, [0, 2, 5, 6])

## Asignación de Tareas

Una forma sencilla y eficiente de realizar la asignación de tareas es realizar una búsqueda de profundidad evitando seguir las ramas incapaces de mejorar

In [5]:
tareas = [[9, 2, 7, 8], [6, 4, 3, 7], [5, 8, 1, 8], [7, 6, 9, 4]]

class Node:
    def __init__(self, inferior, superior, ruta):
        self.inferior = inferior
        self.superior = superior
        self.ruta = ruta
        
    def __gt__(self, other):
        return self.superior > other.superior
    
    def __repr__(self):
        return "<{},{},{}>".format(self.inferior, self.superior, self.ruta)

def RyP_Tareas(tareas):
    max_tareas = max([max(x) for x in tareas])
    m = len(tareas)
    best_ci = 0
    best_cs = max_tareas*m
    
    explorar = [Node(best_ci, best_cs, [])]
    best_node = None
    
    while len(explorar) > 0:
        # Exploramos la rama cuya cota superior sea mayor
        explorar = sorted(explorar, reverse=True)
        #print(explorar)
        
        current = explorar.pop(0)
        n = len(current.ruta)
        #print(current)
        #print()
        
        # Si el nodo ha explorado todo el árbol, comprobar
        # si mejora los resultados almacenados
        if n == m:
            if best_node == None or current > best_node:
                best_node = current
            continue
        
        # No consideramos aquellos nodos cuya cota superior
        # sea menor a la mejor cota inferior encontrada
        if current.superior < best_ci:
            continue
            
        for i,x in enumerate(tareas[n]):
            if i in current.ruta:
                continue
            
            ci = current.inferior + x
            cs = ci + max_tareas * (m - n - 1)
            ruta = current.ruta + [i]
            
            best_ci = max(best_ci, ci)
            best_cs = min(best_cs, cs)
            
            explorar.append(Node(ci, cs, ruta))
    
    # Mostrar relación agente - tarea
    for agent, task in enumerate(best_node.ruta):
        print("{}: job {}".format( chr(65+agent), task+1))
        
RyP_Tareas(tareas)

# En el peor caso, ramificación y poda no permite eliminar de la búsqueda ninguna 
# rama y, por lo tanto, la complejidad del algoritmo es O(A+T) donde A es el 
# número de agentes y T el número de tareas (es decir, es lo mismo que realizar
# una búsqueda por profunidad)

A: job 1
B: job 4
C: job 2
D: job 3
